![title](../images/header.png)

Merging CEO data phase II (2015-2022)
-------
This notebook merges validated data from different CEO CAFI projects
###### For more information contact aurelie.shapiro@fao.org or remi.dannunzio@fao.org

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Set the display option to a large value to prevent text wrapping
pd.set_option('display.max_colwidth', None)

## Enter parameters

In [34]:
#identifier for country 
# COD COG CMR GAB EQG CAF
iso = 'EQG'

### 1. Read all CEO sample files
download your CEO sample files from collect.earth and upload to SEPAL using FileZilla.
add as many files as you have projects

In [35]:
#DRC
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/ceo-CAFI-DDD-RDC-eSBAE-validation-1-sample-data.csv')
#ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/ceo-CAFI-DDD-RDC-eSBAE-validation-2-sample-data.csv')
#ceo3 = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/ceo-CAFI-DDD-RDC-eSBAE-validation-3-sample-data.csv')
#GAB
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/ceo-CAFI-DDD-Gabon-eSBAE-validation-1-sample-data.csv')
#ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/ceo-CAFI-DDD-Gabon-eSBAE-validation-2-sample-data.csv')
#ceo3 = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/ceo-CAFI-DDD-Gabon-eSBAE-validation-3-sample-data.csv')
#ceo4 = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/ceo-CAFI-DDD-Gabon-eSBAE-validation-4-sample-data.csv')
#ceo5 = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/ceo-CAFI-DDD-Gabon-eSBAE-validation-5-sample-data.csv')
#EQG
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/ceo-CAFI-DDD-Equatorial-Guinea-eSBAE-validation-1-sample-data.csv')
#ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/ceo-CAFI-DDD-Equatorial-Guinea-eSBAE-validation-2-sample-data.csv')
ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/ceo-CAFI-DDD-EQG-eSBAE-2023-collection-1-sample-data.csv')
ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/ceo-CAFI-DDD-EQG-eSBAE-2023-collection-2-sample-data.csv')
#COG
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/COG/COG_all_ceo_1522_1622_clean.csv')
#CMR
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/CMR/ceo-CAFI-DDD-Cameroun-eSBAE-validation-1-sample-data.csv')
#ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/CMR/ceo-CAFI-DDD-Cameroun-eSBAE-validation-2-sample-data.csv')
#CAR
#ceo1 = pd.read_csv('/home/sepal-user/module_results/esbae/CAR/ceo-CAFI-DDD-RCA-eSBAE-validation-1-sample-data.csv')
#ceo2 = pd.read_csv('/home/sepal-user/module_results/esbae/CAR/ceo-CAFI-DDD-RCA-eSBAE-validation-2-sample-data.csv')

make a list of the ceo files for processing

In [36]:
#depends on how many files you have
# if you have 5 files
#ceo_files = [ceo1, ceo2, ceo3, ceo4, ceo5]
# if you have 3 files
#ceo_files = [ceo1, ceo2, ceo3]
# if you have 2 files
ceo_files = [ceo1,ceo2]
# if you have 1 file
#ceo_files = [ceo1]

In [37]:
# output file  CEO validated data file with select columns
ceo_data_out =  '/home/sepal-user/module_results/esbae/EQG/EQG_all_ceo_2023_clean.csv'

### end of parameters

In [38]:
column_names = ceo1.columns.tolist()
# Print the list of column names
print(column_names)

['plotid', 'sampleid', 'sample_internal_id', 'lon', 'lat', 'email', 'flagged', 'collection_time', 'analysis_duration', 'imagery_title', 'imagery_attributions', 'sample_geom', 'pl_index', 'pl_chgprob', 'pl_maxforprob', 'pl_strata', 'forêt ou non-forêt en 2022', "Y'a t'il de la régéneration", 'Changement en 2023', 'Type de Changement 2023', 'Exploitation forestière artisanale (<5ha)', 'Activité minière artisanale', 'Infrastructure routière', 'Infrastructure d’urbanisation ou habitations', 'Exploitation forestière industrielle (>5ha)', 'Autre moteur', 'Descriptif autre moteur', 'Activité minière industrielle', 'Agriculture industrielle', 'Agriculture paysanne', 'Année de Changement', 'Type de non-forêt en 2022', 'Type de forêt en 2022', 'Commentaires']


##### add a column named "interpreted" which = 1 when the point has been validated in CEO, otherwise 0
count the number of validated points per ceo file
replace forest and non-forest types with codes

In [70]:
# List to store the modified DataFrames
modified_ceo_files = []

# Initialize collection counter
collection_counter = 1

for i, ceo in enumerate(ceo_files):
       
    # Sort the DataFrame by 'date' in descending order
    ceo.sort_values(by='collection_time', ascending=True, inplace=True)

    # Remove duplicates keeping the last occurrence (the most recent date)
    ceo.drop_duplicates(subset='plotid', keep='first', inplace=True)

    # Reset the index
    ceo.reset_index(drop=True, inplace=True)
    
    # Add 'collection' column with an increasing number
    ceo['collection'] = collection_counter
    
    # Add 'interpreted' column with 1 when 'email' is not null, otherwise 0
    ceo['interpreted'] = np.where(ceo['email'].notna(), 1, 0)
    
    # Calculate total count for each 'interpreted' value
    interpreted_counts = ceo['interpreted'].value_counts().to_dict()
    
    # Append the modified DataFrame to the list
    modified_ceo_files.append(ceo)
    
    # Print count for the current DataFrame
    print(f"ceo {i + 1} # of validated points: {interpreted_counts.get(1, 0)}, # of not validated points: {interpreted_counts.get(0, 0)}")
    
    # Increment the collection counter
    collection_counter += 1

# Concatenate the modified DataFrames into one merged DataFrame
merged_ceo = pd.concat(modified_ceo_files, ignore_index=True)


# Specify the columns you want to keep in the merged DataFrame
columns_to_keep = ['plotid', 'sampleid', 'lon', 'lat', 
                    'sample_geom', 'forêt ou non-forêt en 2022', "Y'a t'il de la régéneration", 
                    'Changement en 2023', 'Type de Changement 2023', 'Exploitation forestière artisanale (<5ha)', 
                    'Activité minière artisanale', 'Infrastructure routière', 'Infrastructure d’urbanisation ou habitations', 
                    'Exploitation forestière industrielle (>5ha)', 'Autre moteur', 'Descriptif autre moteur', 'Activité minière industrielle', 
                    'Agriculture industrielle', 'Agriculture paysanne', 'Année de Changement', 'Type de non-forêt en 2022', 'Type de forêt en 2022', 'collection', 'interpreted']

# Select only the specified columns
merged_ceo = merged_ceo[columns_to_keep]

# Create a dictionary to map old column names to new column names
column_mapping = {
    'forêt ou non-forêt en 2022': 'Ref_FNF_2022', 
    "Y'a t'il de la régéneration": 'Ref_Regeneration',
    'Changement en 2023': 'Ref_Change_2023', 
    'Type de Changement 2023': 'Ref_Change_Type_2023', 
    'Exploitation forestière artisanale (<5ha)': 'ArtFor',
    'Activité minière artisanale': 'ArtMine',
    'Infrastructure routière': 'InfraR', 
    'Infrastructure d’urbanisation ou habitations': 'Urb',
    'Exploitation forestière industrielle (>5ha)':'IndFor', 
    'Autre moteur': 'Other', 
    'Descriptif autre moteur': 'Other_Desc', 
    'Activité minière industrielle': 'IndMine', 
    'Agriculture industrielle': 'IndAg', 
    'Agriculture paysanne': 'ArtAg', 
    'Année de Changement': 'Ref_Year_2023', 
    'Type de non-forêt en 2022':'Ref_NFtype_2022', 
    'Type de forêt en 2022':'Ref_Ftype_2022'
    #add other columns as needed
    #,'NOM': 'Province' # DRC province
}

# Use the rename() method to rename the columns
merged_ceo = merged_ceo.rename(columns=column_mapping)

# Define the replacements as a dictionary
replace_dict = {
    'Ref_FNF_2022': {'non-forêt': '0', 'forêt': '1'},
    'Ref_Ftype_2022': {'1 - forêt dense': '1', '2 - forêt dense sèche': '2', '3 - forêt secondaire': '3','4 - forêt claire sèche': '4','7 - forêt mangrove': '7',
                 '8 - forêt marécageuse': '8', '9 - forêt galérie': '9','10 - plantation forestière': '10', '11- savane arborée': '11'},
    'Ref_NFtype_2022': {'12 - savane arbustive': '12', '13 - savane herbacée': '13', '14 - prairie aquatique': '14', '15 - sol nu végétation éparse': '15',
                  '16 - terres cultivées': '16', '17 - zone baties': '17', '18 - eau': '18'},
    'Ref_Change_2023': {'oui':'1','non':'0','':'0'},
    'ArtFor':{'oui':'1','non':'0','':'0'},
    'IndFor':{'oui':'1','non':'0','':'0'},
    'ArtAg':{'oui':'1','non':'0','':'0'},
    'IndAg':{'oui':'1','non':'0','':'0'},
    'ArtMine':{'oui':'1','non':'0','':'0'},
    'IndMine':{'oui':'1','non':'0','':'0'},
    'Urb':{'oui':'1','non':'0','':'0'},
    'InfraR':{'oui':'1','non':'0','':'0'},
    'Other':{'oui':'1','non':'0','':'0'}
}

# Use the replace() method to replace strings in specified columns
merged_ceo.replace(replace_dict, inplace=True)

columns_to_fill = ['Ref_Change_2023','ArtAg','ArtFor','IndFor','IndAg','ArtMine','IndMine','Urb','InfraR','Other']
# Set NaN values in the selected columns to 0
merged_ceo[columns_to_fill] = merged_ceo[columns_to_fill].fillna('0')
merged_ceo['collection'] = 'coll_2023_' + merged_ceo['collection'].astype(str)

ceo 1 # of validated points: 247, # of not validated points: 0
ceo 2 # of validated points: 155, # of not validated points: 342


remove non validated points and remove duplicates

In [71]:
merged_ceo_interpreted = merged_ceo[merged_ceo['interpreted'] == 1]
len(merged_ceo_interpreted)

402

In [72]:
ceo_column_names = merged_ceo_interpreted.columns.tolist()
# Print the list of column names
print(ceo_column_names)

['plotid', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2022', 'Ref_Regeneration', 'Ref_Change_2023', 'Ref_Change_Type_2023', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_2023', 'Ref_NFtype_2022', 'Ref_Ftype_2022', 'collection', 'interpreted']


In [73]:
merged_ceo_interpreted.head()

,plotid,sampleid,lon,lat,sample_geom,Ref_FNF_2022,Ref_Regeneration,Ref_Change_2023,Ref_Change_Type_2023,ArtFor,ArtMine,InfraR,Urb,IndFor,Other,Other_Desc,IndMine,IndAg,ArtAg,Ref_Year_2023,Ref_NFtype_2022,Ref_Ftype_2022,collection,interpreted
0,0,0,10.329144,1.166597,POINT(10.32914355 1.166597144),1,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,3,coll_2023_1,1
1,1,1,10.235449,1.674684,POINT(10.23544926 1.674684268),1,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_2023_1,1
2,2,2,8.765895,3.447330,POINT(8.76589529 3.447329819),1,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_2023_1,1
3,3,3,10.967397,1.110003,POINT(10.96739656 1.110003281),1,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_2023_1,1
4,4,4,10.259973,1.766492,POINT(10.25997327 1.76649209),1,NaN,0,NaN,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,coll_2023_1,1


##### count points in different categories

In [74]:
merged_ceo_interpreted['interpreted'].value_counts(dropna=False)

interpreted
1    402
Name: count, dtype: int64

In [75]:
pd.pivot_table(merged_ceo_interpreted,values='plotid',index=['Ref_Change_2023'],columns=['interpreted'],aggfunc="count")

interpreted,1
Ref_Change_2023,
0,391
1,11


In [76]:
merged_ceo_interpreted['Ref_FNF_2022'].value_counts(dropna=False)

Ref_FNF_2022
1    371
0     31
Name: count, dtype: int64

In [77]:
merged_ceo_interpreted['Ref_Change_Type_2023'].value_counts(dropna=False)

Ref_Change_Type_2023
NaN              391
déforestation      7
dégradation        4
Name: count, dtype: int64

In [78]:
merged_ceo_interpreted['collection'].value_counts(dropna=False)

collection
coll_2023_1    247
coll_2023_2    155
Name: count, dtype: int64

##### create a LC type column

In [61]:
merged_ceo_interpreted['Ref_LCover_2022'] = merged_ceo_interpreted['Ref_Ftype_2022'].fillna(merged_ceo_interpreted['Ref_NFtype_2022'])
merged_ceo_interpreted['Ref_LCover_2022'].value_counts(dropna=False)

/tmp/ipykernel_432/35551601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_ceo_interpreted['Ref_LCover_2022'] = merged_ceo_interpreted['Ref_Ftype_2022'].fillna(merged_ceo_interpreted['Ref_NFtype_2022'])


Ref_LCover_2022
1     304
3      37
2      19
16     10
15      6
14      6
18      5
7       5
9       5
17      3
8       1
12      1
Name: count, dtype: int64

### 2. add new columns for change and year

clean up columns and replace values

In [67]:
def f(x):
  if x['Ref_FNF_2022'] == '0' : return 'NF'
  elif x['Ref_Change_Type_2023'] == 'déforestation': return 'Def'
  elif x['Ref_Change_Type_2023'] == 'dégradation' : return 'Deg'
  else: return 'Stable'

merged_ceo_interpreted['Ref_Change_Type_2023'] = merged_ceo_interpreted.apply(f, axis=1)
merged_ceo_interpreted['Ref_Change_Type_2023'].value_counts()

/tmp/ipykernel_432/1231062504.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_ceo_interpreted['Ref_Change_Type_2223'] = merged_ceo_interpreted.apply(f, axis=1)


Ref_Change_Type_2223
Stable    360
NF         31
Def         7
Deg         4
Name: count, dtype: int64

In [68]:
def y(x):
  if x['Ref_Change_Type_2023'] == 'Def': return 'Def2023'
  elif x['Ref_Change_Type_2023'] == 'Deg' : return 'Deg2023'
  elif x['Ref_Change_Type_2023'] == 'NF' : return 'NF'
  elif x['Ref_Change_Type_2023'] == 'Stable' : return 'Stable'
  else: return ''

merged_ceo_interpreted['Ref_Change_Year_2023'] = merged_ceo_interpreted.apply(y, axis=1)

annual_counts = merged_ceo_interpreted['Ref_Change_Year_2023'].value_counts()
annual_counts = annual_counts.sort_index()
print(annual_counts)

KeyError: 'Ref_Change_Type_2023'

In [24]:
annual_counts.sum()

13147

In [26]:
pd.pivot_table(merged_ceo_interpreted,values='point_id',index=['Ref_Change_Type_2023'],columns=['interpreted'],aggfunc="count",margins=True,
                             margins_name='Total')

interpreted,1,Total
Ref_Change_Type_1522,,
Def,977,977
Deg,275,275
NF,6297,6297
Stable,5598,5598
Total,13147,13147


In [27]:
merged_ceo_interpreted['Def2023'] = np.where(merged_ceo_interpreted['Ref_Change_Year_2023'] == 'Def2023', 1, 0)

In [28]:
merged_ceo_interpreted['Deg2023'] = np.where(merged_ceo_interpreted['Ref_Change_Year_2023'] == 'Deg2023', 1, 0)

In [29]:
merged_ceo_interpreted['Defall'] = np.where(merged_ceo_interpreted['Ref_Change_Type_2023'] == 'Def', 1, 0)
merged_ceo_interpreted['Degall'] = np.where(merged_ceo_interpreted['Ref_Change_Type_2023'] == 'Deg', 1, 0)

In [30]:
merged_ceo_interpreted['Stable'] = np.where(merged_ceo_interpreted['Ref_Change_Year_2023'] == 'Stable', 1, 0)
merged_ceo_interpreted['NF'] = np.where(merged_ceo_interpreted['Ref_Change_Year_2023'] == 'NF', 1, 0)

In [31]:
merged_ceo_column_names = merged_ceo_interpreted.columns.tolist()
# Print the list of column names
print(merged_ceo_column_names)

['plotid', 'sampleid', 'lon', 'lat', 'sample_geom', 'point_id', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype', 'Ref_Ftype', 'collection', 'interpreted', 'Ref_LCover', 'Ref_Change_Year_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF']


In [32]:
merged_ceo_interpreted.head()

,plotid,sampleid,lon,lat,sample_geom,point_id,Ref_FNF_2015,Ref_Regeneration,Ref_Change_1522,Ref_Change_Type_1522,ArtFor,ArtMine,InfraR,Urb,IndFor,Other,Other_Desc,IndMine,IndAg,ArtAg,Ref_Year_1522,Ref_NFtype,Ref_Ftype,collection,interpreted,Ref_LCover,Ref_Change_Year_1522,Def2016,Def2017,Def2018,Def2019,Def2020,Def2021,Def2022,Deg2016,Deg2017,Deg2018,Deg2019,Deg2020,Deg2021,Deg2022,Defall,Degall,Stable,NF
0,1377222,1377222,15.104924,-4.909934,POINT(15.104924 -4.909934),1377222,0,non,0,NF,0,0,0,0,0,0,NaN,0,0,0,NaN,12,NaN,1,1,12,NF,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1379242,1379242,19.432718,2.746142,POINT(19.432718 2.746142),1379242,1,NaN,1,Def,0,0,0,0,0,0,NaN,0,0,1,2017.0,NaN,3,1,1,3,Def2017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,1379991,1379991,19.252191,2.251043,POINT(19.252191 2.251043),1379991,1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,1,1,1,Stable,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1381523,1381523,19.563450,2.562405,POINT(19.56345 2.562405),1381523,1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,1,1,1,Stable,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1382196,1382196,19.495225,2.300299,POINT(19.495225 2.300299),1382196,1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,1,1,1,Stable,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [33]:
print(merged_ceo_interpreted.columns.values)

['plotid' 'sampleid' 'lon' 'lat' 'sample_geom' 'point_id' 'Ref_FNF_2015'
 'Ref_Regeneration' 'Ref_Change_1522' 'Ref_Change_Type_1522' 'ArtFor'
 'ArtMine' 'InfraR' 'Urb' 'IndFor' 'Other' 'Other_Desc' 'IndMine' 'IndAg'
 'ArtAg' 'Ref_Year_1522' 'Ref_NFtype' 'Ref_Ftype' 'collection'
 'interpreted' 'Ref_LCover' 'Ref_Change_Year_1522' 'Def2016' 'Def2017'
 'Def2018' 'Def2019' 'Def2020' 'Def2021' 'Def2022' 'Deg2016' 'Deg2017'
 'Deg2018' 'Deg2019' 'Deg2020' 'Deg2021' 'Deg2022' 'Defall' 'Degall'
 'Stable' 'NF']


In [34]:
merged_ceo_interpreted['Ref_Change_Year_2023'].value_counts(dropna=False)

Ref_Change_Year_1522
NF         6297
Stable     5598
Def2019     173
Def2018     167
Def2020     158
Def2017     157
Def2021     128
Def2016      97
Def2022      97
Deg2016      60
Deg2020      46
Deg2019      40
Deg2022      39
Deg2021      35
Deg2018      32
Deg2017      23
Name: count, dtype: int64

In [35]:
merged_ceo_interpreted.head()

,plotid,sampleid,lon,lat,sample_geom,point_id,Ref_FNF_2015,Ref_Regeneration,Ref_Change_1522,Ref_Change_Type_1522,ArtFor,ArtMine,InfraR,Urb,IndFor,Other,Other_Desc,IndMine,IndAg,ArtAg,Ref_Year_1522,Ref_NFtype,Ref_Ftype,collection,interpreted,Ref_LCover,Ref_Change_Year_1522,Def2016,Def2017,Def2018,Def2019,Def2020,Def2021,Def2022,Deg2016,Deg2017,Deg2018,Deg2019,Deg2020,Deg2021,Deg2022,Defall,Degall,Stable,NF
0,1377222,1377222,15.104924,-4.909934,POINT(15.104924 -4.909934),1377222,0,non,0,NF,0,0,0,0,0,0,NaN,0,0,0,NaN,12,NaN,1,1,12,NF,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1379242,1379242,19.432718,2.746142,POINT(19.432718 2.746142),1379242,1,NaN,1,Def,0,0,0,0,0,0,NaN,0,0,1,2017.0,NaN,3,1,1,3,Def2017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,1379991,1379991,19.252191,2.251043,POINT(19.252191 2.251043),1379991,1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,1,1,1,Stable,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1381523,1381523,19.563450,2.562405,POINT(19.56345 2.562405),1381523,1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,1,1,1,Stable,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1382196,1382196,19.495225,2.300299,POINT(19.495225 2.300299),1382196,1,NaN,0,Stable,0,0,0,0,0,0,NaN,0,0,0,NaN,NaN,1,1,1,1,Stable,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [36]:
merged_ceo_interpreted['ISO'] = iso

In [39]:
print(merged_ceo_interpreted.dtypes)

plotid                    int64
sampleid                  int64
lon                     float64
lat                     float64
sample_geom              object
point_id                  int64
Ref_FNF_2015             object
Ref_Regeneration         object
Ref_Change_1522          object
Ref_Change_Type_1522     object
ArtFor                   object
ArtMine                  object
InfraR                   object
Urb                      object
IndFor                   object
Other                    object
Other_Desc               object
IndMine                  object
IndAg                    object
ArtAg                    object
Ref_Year_1522           float64
Ref_NFtype               object
Ref_Ftype                object
collection                int64
interpreted               int64
Ref_LCover               object
Ref_Change_Year_1522     object
Def2016                   int64
Def2017                   int64
Def2018                   int64
Def2019                   int64
Def2020 

In [40]:
columns_to_convert = ['Def2023','Deg2023','NF','Stable']
merged_ceo_interpreted[columns_to_convert] = merged_ceo_interpreted[columns_to_convert].fillna(0)
merged_ceo_interpreted[columns_to_convert] = merged_ceo_interpreted[columns_to_convert].astype(int)

### 4. export ceo data to one csv file

In [41]:
merged_ceo_interpreted.to_csv(ceo_data_out,index_label='ID')